# Tensorflow MNIST Classifier demo

This notebook contains an end-to-end demostration of Dioptra that can be run on any modern laptop.
Please see the [example README](README.md) for instructions on how to prepare your environment for running this example.

### Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [75]:
EXPERIMENT_NAME = "mnist_fgm"
EXPERIMENT_DESC = "applying the fast gradient sign (FGM) attack to a classifier trained on MNIST"
QUEUE_NAME = 'Tensorflow CPU'
QUEUE_DESC = 'Tensorflow CPU Queue'
MODEL_NAME = "mnist_classifier"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:5000"

In [134]:
# Import packages from the Python standard library
import json
import os
import pprint
import time
import warnings
from IPython.display import display, clear_output
import logging
import structlog
from pathlib import Path

# Filter out warning messages
warnings.filterwarnings("ignore")
structlog.configure(
    wrapper_class=structlog.make_filtering_bound_logger(logging.ERROR),
)

from dioptra.client import connect_json_dioptra_client, connect_response_dioptra_client, select_files_in_directory, select_one_or_more_files
# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
if os.getenv("DIOPTRA_API") is None:
    os.environ["DIOPTRA_API"] = RESTAPI_ADDRESS

def wait_for_job(job, job_name, quiet=False):
    n = 0
    while job['status'] not in ['finished', 'failed']:
        job = client.jobs.get_by_id(job['id'])
        time.sleep(1)
        if not quiet:
            clear_output(wait=True)
            display("Waiting for job." + "." * (n % 3) )
        n += 1
    if not quiet:
        if job['status'] == 'finished':
            clear_output(wait=True)
            display(f'Job finished. Starting "{job_name}" job.')
        else:
            raise Exception("Previous job failed. Please see tensorflow-cpu logs for details.")
    return job

### Dataset

We obtained a copy of the MNIST dataset when we ran `download_data.py` script. If you have not done so already, see [How to Obtain Common Datasets](https://pages.nist.gov/dioptra/getting-started/acquiring-datasets.html).
The training and testing images for the MNIST dataset are stored within the `/dioptra/data/Mnist` directory as PNG files that are organized into the following folder structure,

    Mnist
    ├── testing
    │   ├── 0
    │   ├── 1
    │   ├── 2
    │   ├── 3
    │   ├── 4
    │   ├── 5
    │   ├── 6
    │   ├── 7
    │   ├── 8
    │   └── 9
    └── training
        ├── 0
        ├── 1
        ├── 2
        ├── 3
        ├── 4
        ├── 5
        ├── 6
        ├── 7
        ├── 8
        └── 9

The subfolders under `training/` and `testing/` are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

### Login to Dioptra and setup RESTAPI client

To connect with the endpoint, we will use a client class defined in the `examples/scripts/client.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below.
The client uses the environment variable `DIOPTRA_RESTAPI_URI`, which we configured at the top of the notebook, to figure out how to connect to the Dioptra RESTful API.

In [77]:
#client = connect_response_dioptra_client()
client = connect_json_dioptra_client()

It is necessary to login to the RESTAPI to be able to perform any functions. Here we create a user if it is not created already, and login with it.

In [78]:
try:
    client.users.create(
        username='pluginuser2',
        email='pluginuser2@dioptra.nccoe.nist.gov',
        password='pleasemakesuretoPLUGINthecomputer'
    )
except:
    pass # ignore if user exists already

client.auth.login(
    username='pluginuser2',
    password='pleasemakesuretoPLUGINthecomputer'
)


DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/users  method=POST
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/users HTTP/1.1" 308 257
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/users/ HTTP/1.1" 409 266
DEBUG:dioptra.client.sessions:Response received: status_code=409
DEBUG:dioptra.client.sessions:HTTP error code returned: status_code=409  method=POST  url=http://localhost:5000/api/v1/users/  text={"error": "EntityExistsError", "message": "Conflict - The User with username having value (pluginuser2) is not available.", "detail": {"entity_type": "User", "existing_id": 1, "entity_attributes": {"username": "pluginuser2"}}, "originating_path": "/api/v1/users/?"}

DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/auth/login  method=POST
DEBUG:urllib3

{'username': 'pluginuser2', 'status': 'Login successful'}

In [79]:
try:
    client.users.create(
        username='dioptra-worker',
        email='dw@dioptra.nccoe.nist.gov',
        password='password'
    )
except:
    pass # ignore if user exists already



DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/users  method=POST
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/users HTTP/1.1" 308 257
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/users/ HTTP/1.1" 409 272
DEBUG:dioptra.client.sessions:Response received: status_code=409
DEBUG:dioptra.client.sessions:HTTP error code returned: status_code=409  method=POST  url=http://localhost:5000/api/v1/users/  text={"error": "EntityExistsError", "message": "Conflict - The User with username having value (dioptra-worker) is not available.", "detail": {"entity_type": "User", "existing_id": 2, "entity_attributes": {"username": "dioptra-worker"}}, "originating_path": "/api/v1/users/?"}



### Upload all the entrypoints in the src/ folder

In [202]:
# import from local filesystem
logging.basicConfig(level=logging.DEBUG) # Sets the root logger level

response = client.workflows.import_resources(group_id=1,
                                             source=select_files_in_directory("../extra/", recursive=True),
                                             config_path="dioptra.toml",
                                             resolve_name_conflicts_strategy="overwrite",
                                            )
resources = response["resources"]

train_ep = resources["entrypoints"]["Train"]
fgm_ep = resources["entrypoints"]["FGM"]
patch_gen_ep = resources["entrypoints"]["Patch Generation"]
patch_apply_ep = resources["entrypoints"]["Patch Application"]
predict_ep = resources["entrypoints"]["Predict"]
metrics_ep = resources["entrypoints"]["Metrics"]
defense_ep = resources["entrypoints"]["Defense"]

entrypoints = [train_ep, fgm_ep, patch_gen_ep, patch_apply_ep, predict_ep, metrics_ep, defense_ep]

DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/workflows/resourceImport  method=POST
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/workflows/resourceImport HTTP/1.1" 200 1335
DEBUG:dioptra.client.sessions:Response received: status_code=200


In [203]:
try:
    experiment = client.experiments.create(group_id=1, name=EXPERIMENT_NAME, description=EXPERIMENT_DESC)
except:
    experiment = client.experiments.get(search=f"name:'{EXPERIMENT_NAME}'")["data"][0]

try:
    queue = client.queues.create(group_id=1, name=QUEUE_NAME, description=QUEUE_DESC)
except:
    queue = client.queues.get(search=f"name:'{QUEUE_NAME}'")["data"][0]

experiment_id = experiment['id']
queue_id = queue['id']

client.experiments.entrypoints.create(experiment_id=experiment_id, entrypoint_ids=entrypoints)

for entrypoint in entrypoints:
    client.entrypoints.queues.create(entrypoint_id=entrypoint, queue_ids=[queue_id])

DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/experiments  method=POST
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/experiments HTTP/1.1" 308 269
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/experiments/ HTTP/1.1" 409 307
DEBUG:dioptra.client.sessions:Response received: status_code=409
DEBUG:dioptra.client.sessions:HTTP error code returned: status_code=409  method=POST  url=http://localhost:5000/api/v1/experiments/  text={"error": "EntityExistsError", "message": "Conflict - The entity with name having value (mnist_fgm), and group_id having value (1) is not available.", "detail": {"entity_type": null, "existing_id": 80, "entity_attributes": {"name": "mnist_fgm", "group_id": 1}}, "originating_path": "/api/v1/experiments/?"}

DEBUG:dioptra.client.sessions:Request made: url=http://loc

### Train a new le_net model on MNIST

In [139]:
job_time_limit = '1h'

training_job = client.experiments.jobs.create(
    experiment_id=experiment_id, 
    description=f"training", 
    queue_id=queue_id,
    entrypoint_id=train_ep, 
    values={"epochs":"3"}, 
    timeout=job_time_limit
)

DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/experiments/80/jobs  method=POST
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/experiments/80/jobs HTTP/1.1" 200 1361
DEBUG:dioptra.client.sessions:Response received: status_code=200


### Generate adversarial examples using FGM attack

In [140]:
job_time_limit = '1h'

training_job = wait_for_job(training_job, 'fgm')
fgm_job = client.experiments.jobs.create(
    experiment_id=experiment_id,
    description=f"fgm",
    queue_id=queue_id,
    entrypoint_id=fgm_ep,
    values={"model_name": MODEL_NAME, "model_version": str(-1)}, # -1 means get the latest model
    timeout=job_time_limit
)

'Job finished. Starting "fgm" job.'

DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/experiments/80/jobs  method=POST
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/experiments/80/jobs HTTP/1.1" 200 1418
DEBUG:dioptra.client.sessions:Response received: status_code=200


### Generate patches based on the model and dataset

In [172]:
def find_artifact_by_name(artifact_name, job):
    job = wait_for_job(job, "", quiet=True)
    for job_artifact in job["artifacts"]:
        artifact = client.artifacts.get_by_id(job_artifact["id"])
        print(Path(artifact["artifactUri"]).name, artifact_name)
        if Path(artifact["artifactUri"]).name == artifact_name:
            print("artifact:", artifact)
            return {
                "id": artifact["id"],
                "snapshotId": artifact["snapshot"],
            }
    raise Exception("Could not retrieve artifact")


In [204]:
job_time_limit = '1h'

wait_for_job(training_job, 'patch_gen')
patch_gen_job = client.experiments.jobs.create(
    experiment_id=experiment_id,
    description=f"patch generation",
    queue_id=queue_id,
    entrypoint_id=patch_gen_ep,
    values={
     "model_name": MODEL_NAME,
     "model_version": str(-1), # -1 means get the latest
     "rotation_max": str(180),
     "max_iter": str(50),
     "learning_rate": str(5.0),
    },
    timeout=job_time_limit
)

'Job finished. Starting "patch_gen" job.'

DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/experiments/80/jobs  method=POST
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/experiments/80/jobs HTTP/1.1" 200 1490
DEBUG:dioptra.client.sessions:Response received: status_code=200


### Generate adversarial examples by attaching generated patches to the testing data

In [205]:
job_time_limit = '1h'

training_job = wait_for_job(training_job, 'patch_apply')
patch_gen_job = wait_for_job(patch_gen_job, 'patch_apply')
print( find_artifact_by_name("patch.tar", patch_gen_job))
patch_apply_job = client.experiments.jobs.create(
    experiment_id=experiment_id,
    description=f"patch application",
    queue_id=queue_id,
    entrypoint_id=patch_apply_ep,
    values={
     "model_name": MODEL_NAME, 
     "model_version": str(-1), # -1 means get the latest model
     "patch_scale": str(0.5),
     "rotation_max": str(180),
    }, 
    artifact_values={
        "patch": find_artifact_by_name("patch.tar", patch_gen_job)
    },
    timeout=job_time_limit
)

'Job finished. Starting "patch_apply" job.'

DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/artifacts/1753  method=GET
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "GET /api/v1/artifacts/1753 HTTP/1.1" 200 694
DEBUG:dioptra.client.sessions:Response received: status_code=200
DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/artifacts/1753  method=GET
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "GET /api/v1/artifacts/1753 HTTP/1.1" 200 694
DEBUG:dioptra.client.sessions:Response received: status_code=200
DEBUG:dioptra.client.sessions:Request made: url=http://localhost:5000/api/v1/experiments/80/jobs  method=POST
DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:5000 "POST /api/v1/experiments/80/jobs HTTP/1.1" 200 1542
DEBUG:dioptra.client.sessions:Response received

patch.tar patch.tar
artifact: {'id': 1753, 'snapshot': 2392, 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}, 'user': {'id': 2, 'username': 'dioptra-worker', 'url': '/api/v1/users/2'}, 'createdOn': '2025-07-24T17:00:19.587465+00:00', 'snapshotCreatedOn': '2025-07-24T17:00:19.587544+00:00', 'lastModifiedOn': '2025-07-24T17:00:19.587544+00:00', 'latestSnapshot': True, 'hasDraft': False, 'tags': [], 'description': 'Artifact, patch, generated and stored as part of job, 1752', 'pluginSnapshotId': 2339, 'taskId': 537, 'isDir': False, 'fileSize': 2056, 'fileUrl': '/api/v1/artifacts/1753/contents', 'artifactUri': 'mlflow-artifacts:/0/65c1f92086564b748cfdfd7806630bf7/artifacts/patch/patch.tar', 'job': 1752}
{'id': 1753, 'snapshotId': 2392}
patch.tar patch.tar
artifact: {'id': 1753, 'snapshot': 2392, 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}, 'user': {'id': 2, 'username': 'dioptra-worker', 'url': '/api/v1/users/2'}, 'createdOn': '2025-07-24T17:00:19.587465+0

### Helper functions to submit infer & defend jobs

In [ ]:
def run_job(
    experiment_id: int,
    queue_id: int,
    entrypoint: dict[str, any], 
    description: str,
    previous_job: dict[str, any] = None, 
    model_name: str = None,
    model_version: int = -1,
    args: dict[str, any] = None, 
    artifacts: dict[str, any] = None, 
    job_time_limit: str = '1h'
):
    args = {} if args is None else args
    artifacts = {} if artifacts is None else artifacts

    if previous_job is not None:
        previous_job = wait_for_job(previous_job, previous_job["description"], quiet=False)
    
    if model_name is not None:
        args['model_name'] = model_name 
        args['model_version'] = model_version
    
    job = client.experiments.jobs.create(
        experiment_id=experiment_id,
        description=description,
        queue_id=queue_id,
        entrypoint_id=ep,
        values=args,
        artifact_values=artifacts,
        timeout=job_time_limit
    )
    
    return job


def defend(
    experiment_id: int, 
    queue_id: int, 
    defense_entrypoint: dict[str, any], 
    previous_job: dict[str, any],
    artifact_name: str,
    defense: str = "spatial_smoothing", 
    defense_kwargs: dict[str, any] = None, 
    job_time_limit: str = '1h'
):
    defense_kwargs = {} if defense_kwargs is None else defense_kwargs

    arg_dict = {
        "def_type":defense,
        "defense_kwargs": json.dumps(defense_kwargs)
    }

    artifacts = {
        "dataset" : find_artifact_by_name(artifact_name, previous_job)
    }
    
    defense_job = run_job(
        experiment_id=experiment_id, 
        queue_id=queue_id, 
        entrypoint=defense_entrypoint, 
        description=f"{defense} defense against {previous_job[""]}",
        previous_job=previous_job,
        args=arg_dict,
        artifacts=artifacts,
        job_time_limit=job_time_limit
    )
    
    return defense_job

def predict(
    experiment_id: int,
    queue_id: int,
    predict_entrypoint: dict[str, any],
    previous_job: dict[str, any],
    artifact_name: str,
    job_time_limit='1h'
):
    artifacts = {
        "dataset" : find_artifact_by_name(artifact_name, previous_job)
    }

    predict_job = run_job(
        experiment_id=experiment_id, 
        queue_id=queue_id, 
        entrypoint=predict_entrypoint,
        description=f"{}",
        model_name=MODEL_NAME,
        args=arg_dict,
        artifacts=artifacts,
        previous_job=previous_job, 
        job_time_limit=job_time_limit
    )

def measure(
    experiment_id: int, 
    queue_id: int, 
    measure_ep: dict[str, any], 
    previous_job: dict[str, any], 
    artifact_name: str = "predictions",
    job_time_limit='1h'
):
    artifacts = {
        "predictions" : find_artifact_by_name(artifact_name, previous_job)
    }
    
    metrics_job = run_job(
        experiment_id=experiment_id, 
        queue_id=queue_id, 
        entrypoint=measure_ep,
        previous_job=previous_job,
        args={},
        artifacts=artifacts,
        job_time_limit=job_time_limit
    )
    
    return metrics_job

def get_metrics(job):
    wait_for_job(job, 'metrics', quiet=True)
    return client.jobs.get_metrics_by_id(job_id=job['id'])

### Base Accuracy against FGM without Defenses

In [ ]:
predict_fgm = predict(experiment_id, queue_id, predict_ep, fgm_job, artifact_name="adversarial_dataset")
measure_fgm = measure(experiment_id, queue_id, metrics_ep, predict_fgm)

### Run Spatial Smoothing against FGM

In [ ]:
spatial_job_fgm = defend(experiment_id, queue_id, defense_ep, fgm_job, artifact_name="adversarial_dataset", defense="spatial_smoothing")

In [ ]:
predict_spatial_fgm = predict(experiment_id, queue_id, predict_ep, spatial_job_fgm, artifact_name="defended_dataset")
measure_spatial_fgm = measure(experiment_id, queue_id, metrics_ep, predict_spatial_fgm)

### Run JPEG Compression Defense against FGM

In [ ]:
jpeg_comp_job_fgm = defend(experiment_id, queue_id, defense_ep, fgm_job, artifact_name="adversarial_dataset", defense="jpeg_compression")

In [ ]:
predict_jpeg_comp_fgm = predict(experiment_id, queue_id, predict_ep, jpeg_comp_job_fgm, artifact_name="defended_dataset")
measure_jpeg_comp_fgm = measure(experiment_id, queue_id, metrics_ep, predict_jpeg_comp_fgm)

### Run Gaussian Defense against FGM

In [ ]:
gaussian_job_fgm = defend(experiment_id, queue_id, defense_ep, fgm_job, artifact_name="adversarial_dataset", defense="gaussian_augmentation", defense_kwargs={
        "augmentation": False,
        "ratio": 1,
        "sigma": .1,
        "apply_fit": False,
        "apply_predict": True
    }
)

In [ ]:
predict_gaussian_fgm = predict(experiment_id, queue_id, predict_ep, gaussian_job_fgm, artifact_name="defended_dataset")
measure_gaussian_fgm = measure(experiment_id, queue_id, metrics_ep, predict_gaussian_fgm)

### Run Spatial Smoothing, JPEG Compression, Gaussian Defense against Patch Attack

In [ ]:
predict_patch = predict(experiment_id, queue_id, predict_ep, patch_apply_job, artifact_name="adversarial_dataset")
measure_patch = measure(experiment_id, queue_id, metrics_ep, predict_patch)

In [ ]:
spatial_job_patch = defend(experiment_id, queue_id, defense_ep, patch_apply_job, artifact_name="adversarial_dataset", defense="spatial_smoothing")

In [ ]:
predict_spatial_patch = predict(experiment_id, queue_id, predict_ep, spatial_job_patch, artifact_name="defended_dataset")
measure_spatial_patch = measure(experiment_id, queue_id, metrics_ep, predict_spatial_patch)

In [ ]:
jpeg_comp_job_patch = defend(experiment_id, queue_id, defense_ep, patch_apply_job, artifact_name="adversarial_dataset", defense="jpeg_compression")

In [ ]:
predict_jpeg_comp_patch = predict(experiment_id, queue_id, predict_ep, jpeg_comp_job_patch, artifact_name="defended_dataset")
measure_jpeg_comp_patch = measure(experiment_id, queue_id, metrics_ep, predict_jpeg_comp_patch)

In [ ]:
gaussian_job_patch = defend(experiment_id, queue_id, defense_ep, patch_apply_job, artifact_name="adversarial_dataset", defense="gaussian_augmentation", defense_kwargs={
        "augmentation": False,
        "ratio": 1,
        "sigma": .1,
        "apply_fit": False,
        "apply_predict": True
    }
)

In [ ]:
predict_gaussian_patch = predict(experiment_id, queue_id, predict_ep, gaussian_job_patch, artifact_name="defended_dataset")
measure_gaussian_patch = measure(experiment_id, queue_id, metrics_ep, predict_gaussian_patch)

### Retrieve and Display Metrics from Dioptra

In [ ]:
import pprint

metrics = {
    "trained": get_metrics(training_job),
    "fgm": get_metrics(measure_fgm),
    "patch": get_metrics(measure_patch),
    "jpeg_fgm": get_metrics(measure_jpeg_comp_fgm),
    "spatial_fgm": get_metrics(measure_spatial_fgm),
    "gaussian_fgm": get_metrics(measure_gaussian_fgm),
    "jpeg_patch": get_metrics(measure_jpeg_comp_patch),
    "spatial_patch": get_metrics(measure_spatial_patch),
    "gaussian_patch": get_metrics(measure_gaussian_patch)
}

pp = pprint.PrettyPrinter(depth=4)
pp.pprint(metrics)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

scenarios = {
    'trained': 'Base Model',
    'fgm': 'Fast Gradient Method (Attack)',
    'jpeg_fgm': 'JPEG Compression vs. FGM (Defense)',
    'spatial_fgm': 'Spatial Smoothing vs. FGM (Defense)',
    'gaussian_fgm': 'Gaussian Noise vs. FGM (Defense)',
    'patch': 'Adversarial Patch (Attack)',
    'jpeg_patch': 'JPEG Compression vs. Patch (Defense)',
    'spatial_patch': 'Spatial Smoothing vs. Patch (Defense)',
    'gaussian_patch': 'Gaussian Noise vs. Patch (Defense)'
}
names = [scenarios[k] for k in scenarios.keys()]
values = [[job_metric['value'] * 100 for job_metric in metrics[k] if job_metric['name'] == 'accuracy'][0] for k in scenarios.keys()]

fig, ax = plt.subplots(figsize =(16, 9))

# Horizontal Bar Plot
ax.barh(names, values)

# Add padding between axes and labels
ax.xaxis.set_tick_params(pad = 5)
ax.yaxis.set_tick_params(pad = 10)

# Show top values 
ax.invert_yaxis()

# Add annotation to bars
for i in ax.patches:
    plt.text(i.get_width()+0.2, i.get_y()+0.5, 
             str(round((i.get_width()), 2)),
             fontsize = 10, fontweight ='bold',
             color ='grey')

# Add Plot Title
ax.set_title('Inference Percent Accuracy',
             loc ='left', )

# Show Plot
plt.show()